# setGeoID

Input: compiled_data.csv

Output: sdData.csv

Task: Convert the primary key to GeoID (stored in the first column of the input file)

In [89]:
# Initialize variables
input_filename = "data/compiled_data.txt"
output_filename = 'data/sdData.csv'

In [90]:
str(input_filename)

'data/compiled_data.txt'

In [91]:
# create functions (if needed)
def decimal_point_exists(s):
    return "." in s

# used to convert varying length census tract numbers to 6-digit tract numbers based on the rule described in the function
def format_number(num):
    num_str = str(float(num))
    integer_part, decimal_part = num_str.split('.')
    integer_part_padded = integer_part.zfill(4)
    return integer_part_padded + decimal_part.ljust(2, '0')

# Example usage
test = format_number(54.5)
print(test)
print(format_number(12.3))    # Output: '001230'
print(format_number(1))       # Output: '000100'
print(format_number(4556.3))  # Output: '455630'

005450
001230
000100
455630


## Load FIPS codes into a 2D array 

In [92]:
count2 = 0
# empty 2D array with 2 rows
FIPScodes = [[] for _ in range(2)]

with open('data\\county_FIPScodes.txt', 'r') as input_file:
    for line in input_file:
        count2 = count2 + 1;
        words = line.strip().split()
        #print(words)
        if (count2%2 == 0):
            if (len(words) == 4):
                FIPScodes[0].append(words[-4] + ' ' + words[-3])
            else:
                FIPScodes[0].append(words[-3])
            FIPScodes[1].append(words[-1])
        # this gives the county name and code of the 3rd county
        # FIPScodes[0][3], FIPScodes[1][3]

In [93]:
len(FIPScodes), FIPScodes[0][3], FIPScodes[1][3]

(2, 'Benton', '007')

FIPScodes[0][3], FIPScodes[1][3] gives county name and tract number respectively. Note block group is always a single digit number so there is no need for translation for block group number. 

## Import data

The following cell imports the data into different hard-coded arrays. This can be a generalized later. 

In [94]:
count = 0
err = 0
blocks = []
census_tracts = []
counties = [] # updated to counties_FIPS later (use counties_FIPS)
population_den = []
area = []
female_per = []
children_living_with_single_parents = []
owner_occupied = []
some_college_more = []
dropped_out_schoolOrCollege = []
sep_wid_div = []
unemp = []
debt_housing_units = []
housing_retirement_income = []
females_per_tier1_jobs = []
household_income_less_than_20k = []
household_size_greater_than_equal_3 = []
age_35_to_54 = []
family_household = []
household_self_emp = []
hh_dividends = []
yr_built = []
electricity_cost_per = []
gross_rent = []
pool_publicTransport = []
bike = []

with open(input_filename, 'r') as input_file:
    for line in input_file:
        count = count + 1
        words = line.strip().split()
        if len(words) < 32:
                print(words)
                print('count: ')
                print(count)
        if count != 1:
            # blocks.append(words[1])
            blocks.append(words[2].replace(',', ''))
            census_tracts.append(words[5].replace(',', ''))
            counties.append(words[6].replace(',', ''))
            
            population_den.append(words[9])
            area.append(words[10])
            female_per.append(words[11])
            children_living_with_single_parents.append(words[12])
            owner_occupied.append(words[13])
            some_college_more.append(words[14])
            dropped_out_schoolOrCollege.append(words[15])
            sep_wid_div.append(words[16])
            unemp.append(words[17])
            debt_housing_units.append(words[18])
            housing_retirement_income.append(words[19])
            females_per_tier1_jobs.append(words[20])
            household_income_less_than_20k.append(words[21])
            household_size_greater_than_equal_3.append(words[22])
            age_35_to_54.append(words[23])
            family_household.append(words[24])
            household_self_emp.append(words[25])
            hh_dividends.append(words[26])
            yr_built.append(words[27])
            electricity_cost_per.append(words[28])
            gross_rent.append(words[29])
            pool_publicTransport.append(words[30])
            bike.append(words[31])  

print("length of blocks = ", len(blocks), ", of tracts = ", len(census_tracts), ", of counties = ", len(counties))

['Unnamed:', '0', 'population_den', 'Area', 'female_per', 'children_living_with_single_parents', 'owner_occupied', 'some_college_more', 'dropped_out_schoolOrCollege', 'sep_wid_div', 'unemp', 'debt_housing_units', 'housing_retirement_income', 'females_per_tier1_jobs', 'household_income_less_than_20k', 'household_size_greater_than_equal_3', 'age_35_to_54', 'family_household', 'household_self_emp', 'hh_dividends', 'yr_built', 'electricity_cost_per', 'gross_rent', 'pool_publicTransport', 'bike']
count: 
1
length of blocks =  2294 , of tracts =  2294 , of counties =  2294


In [95]:
counties_FIPS = [None] * len(counties)
for i, county in enumerate(counties):
    for j in range(len(FIPScodes[0])):
        if (county == FIPScodes[0][j]):
            counties_FIPS[i] = FIPScodes[1][j]

In [96]:
key_AR = []

for i in range(len(blocks)):
    elem3 = blocks[i]
    elem2 = census_tracts[i]
    # print some random data before and after formatting to see the effect of format number function
    if i == 67:
        print(elem2)
    elem2 = format_number(elem2)
    if i == 67:
        print(elem2)
 
    elem1 = str('05') + str(counties_FIPS[i]) # append 05 for Arkansas
    result_elem = f"{elem1}-{elem2}-{elem3}"
    key_AR.append(result_elem)

print(key_AR[67])

201.02
020102
05007-020102-4


In [97]:
# combine the arrays
result_array = []

for i in range(len(key_AR)):
    concatenated_string = f"{key_AR[i]}, {population_den[i]}, {area[i]}, {female_per[i]},\
 {children_living_with_single_parents[i]}, {owner_occupied[i]},\
 {some_college_more[i]}, {dropped_out_schoolOrCollege[i]}, {sep_wid_div[i]}, {unemp[i]},\
 {debt_housing_units[i]}, {housing_retirement_income[i]},\
 {females_per_tier1_jobs[i]}, {household_income_less_than_20k[i]}, {household_size_greater_than_equal_3[i]},\
 {age_35_to_54[i]}, {family_household[i]}, {household_self_emp[i]}, {hh_dividends[i]}, {yr_built[i]},\
 {electricity_cost_per[i]}, {gross_rent[i]}, {pool_publicTransport[i]}, {bike[i]}"
    
    result_array.append(concatenated_string)
"""population_den
area
female_per
children_living_with_single_parents
owner_occupied
some_college_more
dropped_out_schoolOrCollege
sep_wid_div
unemp
debt_housing_units
housing_retirement_income
females_per_tier1_jobs
household_income_less_than_20k
household_size_greater_than_equal_3
age_35_to_54
family_household
household_self_emp
hh_dividends
yr_built
electricity_cost_per
gross_rent
pool_publicTransport
bike"""
print("length = ", len(result_array), ", ex: ", result_array[7])
for i in range(len(result_array)):
    if i > 140 and i < 160:
        print("i = ", i, "cell = ", result_array[i])

length =  2294 , ex:  05001-480500-2, 2476.3, 0.75, 0.52, 0.96, 262, 0.54, 0, 0.2, 0.03, 0.68, 0.27, 0.46, 0.79, 0.25, 0.25, 0.65, 0.08, 0.04, 1966, 0.53, 0.16, 0.05, 0
i =  141 cell =  05007-020805-1, 803.4, 1.89, 0.47, 0.04, 562, 0.78, 0, 0.09, 0, 0.71, 0.32, 0.42, 0.96, 0.35, 0.21, 0.9, 0.04, 0.33, 1999, 0.71, 0, 0.04, 0
i =  142 cell =  05007-020805-2, 490.8, 3.84, 0.42, 0.14, 564, 0.69, 0, 0.29, 0.06, 0.84, 0.27, 0.38, 0.88, 0.25, 0.25, 0.58, 0.2, 0.28, 1980, 0.59, 0.06, 0.06, 0
i =  143 cell =  05007-020805-3, 1118.7, 0.97, 0.51, 0.18, 435, 0.69, 0, 0.1, 0.02, 0.5, 0.33, 0.41, 0.99, 0.24, 0.19, 0.78, 0.12, 0.45, 1985, 0.73, 0.22, 0.16, 0
i =  144 cell =  05007-020806-1, 761.7, 1.32, 0.55, 0, 484, 0.78, 0, 0.22, 0.02, 0.5, 0.64, 0.26, 0.97, 0.11, 0.07, 0.64, 0.02, 0.31, 1993, 0.79, 0, 0.04, 0
i =  145 cell =  05007-020806-2, 219.4, 8.42, 0.46, 0.09, 602, 0.52, 0, 0.28, 0.01, 0.47, 0.24, 0.27, 0.88, 0.35, 0.21, 0.49, 0.02, 0.23, 2001, 0.75, 0, 0.02, 0
i =  146 cell =  05007-020806-

In [100]:
import csv
count = 0
# heading = ["key_AR", "population_den", "area", "male_per", "female_per", "household_size", "children_living_with_single_parents", "owner_occupied", "two_or_more_vehicles_per_household", "some_college_more", "dropped_out_schoolOrCollege", "sep_wid_div", "unemp", "debt_housing_units", "housing_retirement_income", "males_per_tier1_jobs", "females_per_tier1_jobs", "household_income_less_than_20k", "household_size_greater_than_equal_3"]
# removes quotation marks
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
    if count == 0:
        file.write("key,population_den,area,female_per,children_living_with_single_parents,owner_occupied,some_college_more,dropped_out_schoolOrCollege,sep_wid_div,unemp,debt_housing_units,housing_retirement_income,females_per_tier1_jobs,household_income_less_than_20k,household_size_greater_than_equal_3,age_35_to_54,family_household,household_self_emp,hh_dividends,yr_built,electricity_cost_per,gross_rent,pool_publicTransport,bike\n")
    for row in result_array:
        writer.writerow(row.split(', '))